In [ ]:
import json
import numpy as np
import pandas as pd
import plotly.express as px

## Открываем файл

In [ ]:
with open("imopse_parsed/200_20_145_15.json") as file:
    data = json.load(file)

In [ ]:
def keys_to_int(d):
    return {int(key): value for key, value in d.items()}

info = data["info"]
N_TASKS = info["n_tasks"]
N_TASKS_PLUS_DUMMY= N_TASKS + 2
N_SKILLS = info["n_skill_types"]
N_WORKERS = info["n_workers"]
N = int(N_TASKS)

worker_pays = keys_to_int(data["worker_pays"])
worker_skill_vectors = keys_to_int(data["worker_skill_vectors"])
durations = keys_to_int(data["tast_durations"])
task_skill_vectors = keys_to_int(data["task_skill_vectors"])
predecessors = keys_to_int(data["task_depends_from"])

In [ ]:
# Горизонт планирования
HORIZON = 10000

In [ ]:
# Создаем матрицу, где a[i][j] показывает,
# можем ли назначить сотрудника i на работу j

def no_skill_issue(worker_id, task_id):
    a = worker_skill_vectors[worker_id]
    b = task_skill_vectors[task_id]
    return (np.array(a) >= np.array(b)).all()

can_worker_do_task = np.array([
    [
        no_skill_issue(worker_id=worker_id, task_id=task_id) 
        for task_id in durations.keys()
    ]
    for worker_id in worker_pays.keys()
])

In [ ]:
np.random.seed(0)

# Индивидуальные календари
# id сотрудника: список (работает ли в i день)
random_calendar = {
    i: np.random.choice([0, 1], p=[0.1, 0.9], size=HORIZON)
    for i in worker_pays.keys()
}

## SGS

In [ ]:
# Менеджер назначений
# Проверяет назначение ресурсов на задачи

class AssignmentManager:
    
    def __init__(self, horizon, n_workers, is_avaliable):
        self.horizon = horizon
        self.n_workers = n_workers
        self.is_busy = np.zeros((N_WORKERS, horizon), dtype=np.int8)
        
        self.is_avaliable = is_avaliable
                
    def check_is_avaliable(self, worker, start, finish):
        answer = (self.is_busy[worker][start:finish] == 0).all()
        return answer
    
    def allocate(self, worker, start, finish):
        if not self.check_is_avaliable(worker, start, finish):
            raise ValueError()    
        #self.is_busy[worker][start:finish] = 1
        self.is_busy[worker][start:finish] += 1
        
    def find_earliest(self, worker, duration, starting_from):
        """
        Находим, куда можно вставить работу
        """
        work_left = int(duration)
        job_start_time = None
        for t in range(starting_from, self.horizon+1):            
            if work_left == 0:
                return (job_start_time, t)
            
            if self.is_busy[worker][t]:
                work_left = int(duration)
                job_start_time = None
                continue
        
            if not self.is_avaliable[worker][t]:
                continue
            
            work_left -= 1
            if job_start_time is None:
                job_start_time = t

In [ ]:
# Менеджер зависимостей
# Проверяет ограничения на зависимости
# (пока только тип конец-начало)

class PrecedenceManager:
    
    def __init__(self, predecessors):
        self.predecessors = predecessors
        self.is_completed = {
            i: False 
            for i in predecessors.keys()
        }
        
    def can_start(self, job_id):
        predecessors = self.predecessors[job_id]
        return all(
            self.is_completed[i]
            for i in predecessors
        )
    
    def what_can_start(self):
        return [
            i for i in self.predecessors.keys()
            if self.can_start(i) and not self.is_completed[i]
        ]
    
    def set_complete(self, job_id):
        self.is_completed[job_id] = True
        
    def are_all_completed(self):
        return all(self.is_completed.values())

In [ ]:
class ProjectPlan:
    
    def __init__(self, predecessors, n_workers, durations):
        self.durations = durations
        self.n_workers = n_workers
        self.am = AssignmentManager(horizon=HORIZON, n_workers=N_WORKERS, is_avaliable=random_calendar)
        self.pm = PrecedenceManager(predecessors)
        self.gantt_chart = {
            i: None
            for i in predecessors.keys()
        }
        
    def add_to_schedule(self, job, start, finish):
        self.pm.set_complete(job)
        self.gantt_chart[job] = (start, finish)
    
    def ssgs(self, activity_list, assignment_list):
        """
        Алгоритм Serial Schedule Generation Scheme
        """
        
        self.pm.set_complete(0)
        self.gantt_chart[0] = (0, 0)
        order_added = [0]
        
        while not self.pm.are_all_completed():
            candidates = self.pm.what_can_start()
            selected = max(candidates, key=lambda a: activity_list[a])
            order_added.append(selected)

            when_predecessors_complete = max(
                self.gantt_chart[i][1]
                for i in self.pm.predecessors[selected] or [0]
            )
    
            start, finish = self.am.find_earliest(
                worker=assignment_list[selected], 
                duration=self.durations[selected]+1, 
                starting_from=when_predecessors_complete+1
            )
            self.am.allocate(assignment_list[selected], start, finish)
            
            self.add_to_schedule(selected, start, finish)
            
        return self.get_metrics()
    
    def get_metrics(self):
        time = max(i[1] for i in self.gantt_chart.values())
        amount_worked = self.am.is_busy.sum(axis=1)
        cost = sum(
            amount_worked[i] * worker_pays[i] 
            for i in range(N_WORKERS)
        )
        n_who_worked = (amount_worked > 0).sum()
        return dict(
            time=time,
            cost=cost,
            n_who_worked=n_who_worked
        )
    
    def restart(self):
        # TODO
        pass

In [ ]:
def get_score(activity_list, assignment_list, metric="time"):
    project = ProjectPlan(
        predecessors=predecessors,
        n_workers=N_WORKERS,
        durations=durations, 
    )
    assert is_assignment_valid(assignment_list)
    
    if metric == "weighted":
        metrics = project.ssgs(activity_list, assignment_list)
        return (
            metrics["time"] + 
            metrics["cost"] // 100_000
        )
    
    return project.ssgs(activity_list, assignment_list)[metric]

## Optimize

In [ ]:
# Пример случайного решения

activity_list = {
    i: np.random.rand()
    for i in predecessors.keys()
}
assignment_list = {
    i: np.random.randint(N_WORKERS)
    for i in predecessors.keys()
}

# Чиним неправильные назначения
new_assignment = {}
for task_id, worker_id in assignment_list.items():
    if not can_worker_do_task[worker_id][task_id]:
        candidates = np.arange(N_WORKERS)[can_worker_do_task[:, task_id]]
        new_assignment[task_id] = np.random.choice(candidates)
    else:
        new_assignment[task_id] = worker_id
assignment_list = dict(new_assignment)

def is_assignment_valid(assignment_list):
    return all(
        can_worker_do_task[worker_id][task_id]
        for task_id, worker_id in assignment_list.items()
    )
is_assignment_valid(assignment_list)

In [ ]:
# Считаем baseline - случайный поиск (250 случайных примеров)

random_solutions = []
for i in range(250):
    
    activity_list = {
        i: np.random.rand()
        for i in predecessors.keys()
    }
    assignment_list = {
        i: np.random.randint(N_WORKERS)
        for i in predecessors.keys()
    }

    new_assignment = {}
    for task_id, worker_id in assignment_list.items():
        if not can_worker_do_task[worker_id][task_id]:
            candidates = np.arange(N_WORKERS)[can_worker_do_task[:, task_id]]
            new_assignment[task_id] = np.random.choice(candidates)
        else:
            new_assignment[task_id] = worker_id
    assignment_list = dict(new_assignment)

    time = get_score(activity_list, assignment_list, metric="cost")
    random_solutions.append(time)

baseline = min(random_solutions)
print(baseline)    

In [ ]:
class AntSolution:
    """All about a solution: different representations and fitness"""
    
    def __init__(self, priority_pairs, assignment_ints):
        self.priority_pairs = priority_pairs
        random_key = np.array([1000 * i[0] + i[1] for i in priority_pairs])  # maybe fix later
        self.random_key = random_key
        self.assignment_ints = assignment_ints
        self.fitness = get_score(
            {i: value for i, value in enumerate(random_key)},
            {i: value for i, value in enumerate(assignment_ints)},
            metric="cost"
        )

In [ ]:
class Pheromones:
    
    def __init__(self, n_bins, n_subbins, evaporation_rate=0.25):
        self.pheromone_matrix = np.random.uniform(10, 100, (N_TASKS_PLUS_DUMMY, n_bins))
        self.subpheromone_matrix = np.random.uniform(10, 100, (N_TASKS_PLUS_DUMMY, n_bins, n_subbins))
        self.apheromone_matrix = np.random.uniform(10, 100, (N_TASKS_PLUS_DUMMY, N_WORKERS))
        self.evaporation_rate = evaporation_rate
        
    def get_pheromones(self):
        return (
            self.pheromone_matrix,
            self.subpheromone_matrix,
            self.apheromone_matrix
        )
    
    def update_pheromones(self, new_pheromones, new_subpheromones, new_apheromones):
        self.pheromone_matrix = (
            self.pheromone_matrix * (1 - self.evaporation_rate) + 
            new_pheromones * self.evaporation_rate
        )
        self.subpheromone_matrix = (
            self.subpheromone_matrix * (1 - self.evaporation_rate) +
            new_subpheromones * self.evaporation_rate
        )
        self.apheromone_matrix = (
            self.apheromone_matrix * (1 - self.evaporation_rate) +
            new_apheromones * self.evaporation_rate
        )

In [ ]:
class History:
    """Tracking generations and stats"""
    
    def __init__(self):
        self.generations = []
        self.current_generation = []
    
    def add_solution(self, solution):
        self.current_generation.append(solution)
        
    def end_generation(self):
        self.generations.append(self.current_generation)
        self.current_generation = []
        
    def get_stats(self):
        mins = [
            min(solution.fitness for solution in generation)
            for generation in self.generations
        ]
        mean = [
            sum(solution.fitness for solution in generation) / len(generation)
            for generation in self.generations
        ]
        return (mins, mean)
    
    def get_generation_stats(self):
        return (
            [i.fitness for i in self.current_generation],
            [i.priority_pairs for i in self.current_generation],
            [i.assignment_ints for i in self.current_generation]
        )

In [ ]:
class AntColony:
    
    def __init__(self, n_per_iteration=20, n_bins=5, n_subbins=5):
        
        self.n_bins = n_bins
        self.n_subbins = n_subbins
        self.n_per_iteration = n_per_iteration
        
        self.pheromones = Pheromones(n_bins, n_subbins)
        self.history = History()
    
    @staticmethod
    def weighted_choice(array, weights):
        probabilities = weights / sum(weights)
        return np.random.choice(array, p=probabilities)
    
    @staticmethod
    def split_to_bins(array):
        bins = np.quantile(array, np.linspace(1, 0, 6))
        bins[0] = np.inf  # small fix for borders
        return np.digitize(array, bins=bins)
    
    def get_one_solution(self):
        """Get one solution based on current pheromones"""
        
        pheromone_matrix, subpheromone_matrix, assignment_matrix = self.pheromones.get_pheromones()
        order = []
        workers = []
        for i in range(0, N_TASKS_PLUS_DUMMY):
            chosen_bin = self.weighted_choice(
                np.arange(self.n_bins), 
                pheromone_matrix[i]
            )
            chosen_subbin = self.weighted_choice(
                np.arange(self.n_subbins), 
                subpheromone_matrix[i][chosen_bin]
            )
            is_fit = np.array([can_worker_do_task[w][i] for w in range(N_WORKERS)])
            chosen_worker = self.weighted_choice(
                np.arange(N_WORKERS), 
                assignment_matrix[i] * is_fit
            )
            order.append([chosen_bin, chosen_subbin])
            workers.append(chosen_worker)
        return AntSolution(np.array(order), np.array(workers))
    
    def get_one_generation(self):
        """Get pheromones from one generation"""
        
        for i in range(self.n_per_iteration):
            solution = self.get_one_solution()
            self.history.add_solution(solution)
                
        fitness_list, priority_pairs_list, assignment_list = self.history.get_generation_stats()
        total_pheromone, total_subpheromone, total_apheromone = \
            self.calculate_pheromones(fitness_list, priority_pairs_list, assignment_list)
        
        self.history.end_generation()
        return (total_pheromone, total_subpheromone, total_apheromone)
    
    def calculate_pheromones(self, fitness_list, priority_pairs_list, assignment_list):
        """Get amount of pheromones to add"""
        
        total_pheromone = np.zeros([N_TASKS_PLUS_DUMMY, self.n_bins])
        total_subpheromone = np.zeros([N_TASKS_PLUS_DUMMY, self.n_bins, self.n_subbins])
        total_apheromone = np.zeros([N_TASKS_PLUS_DUMMY, N_WORKERS])
        
        lengths_digitized = self.split_to_bins(fitness_list)
        for quantile_n, activity_list, assignment in zip(lengths_digitized, priority_pairs_list, assignment_list):
            pheromone_amount = (quantile_n)**2 * np.sign(quantile_n)
            for node, value in enumerate(activity_list):
                total_pheromone[node, value[0]] += pheromone_amount
                total_subpheromone[node, value[0], value[1]] += pheromone_amount
            for node, value in enumerate(assignment):
                total_apheromone[node, value] += pheromone_amount
        
        return (
            np.clip(total_pheromone, 0, 10**6), 
            np.clip(total_subpheromone, 0, 10**6),
            np.clip(total_apheromone, 0, 10**6)
        )
    
    def update_pheromones(self, update_subpheromones=True):
        """Update pheromones"""
        new_pheromone, new_subpheromone, new_apheromone = self.get_one_generation()
        self.pheromones.update_pheromones(new_pheromone, new_subpheromone, new_apheromone)

In [ ]:
np.random.seed(0)

colony = AntColony(n_per_iteration=25, n_bins=4, n_subbins=4)

for j in range(10):

    for i in range(10):    
        colony.update_pheromones(update_subpheromones=False)
        print("-", end="")
        
    for i in range(10):    
        colony.update_pheromones(update_subpheromones=True)
        print("=", end="")
    
    print("#")

In [ ]:
mins, mean = colony.history.get_stats()

fig = px.scatter(y=[
    pd.Series(mins).cummin(), 
    mean,
    [baseline]*len(mins)
])

fig.update_layout(showlegend=True,
                  template="plotly_white",
                  xaxis_title='Generation',                   
                  yaxis_title='Metric')
fig.update_traces(mode='lines', line_width=4)
fig.data[0].line.color = "black"
fig.data[1].line.color = "green"
fig.data[2].line.color = "lightgrey"

fig.data[0].name = "Best"
fig.data[1].name = "Mean"
fig.data[2].name = "Baseline"

fig.update_layout(title="Target: wage", legend=dict(
    yanchor="top",
    y=0.50,
    xanchor="right",
    x=0.95
))
fig.update_layout(height=1080, width=1920, font_size=24)
fig.show()

In [ ]:
# last_pheromones, last_subpheromones, last_apheromones = colony.pheromones.get_pheromones()

# fig = px.imshow(
#     last_pheromones,
#     color_continuous_scale=['black', 'darkred', 'red', 'orange', 'yellow', 'lime'])
# fig.update_layout(template='plotly_dark', height=2000, width=400, coloraxis_showscale=False)
# fig.show()